In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras import  Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Dropout, Dense, BatchNormalization, LSTM
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
import bnci_utils as utils

In [2]:
num_participants = 18
dataset_path = os.path.join('dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1))
         for i in range(num_participants)] # P01 - P18 files


# Set seed to produce consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# Function to transform the dataset to be usable for the neural network - i.e one hot encode and reshape the dataset
def transform_dataset(features, labels):
    labels = labels.reshape((-1, 1)) # reshape so one hot encoding can be used
    labels = OneHotEncoder().fit_transform(labels).toarray() # apply one hot encoding
    features = features.reshape((features.shape[0], 14, -1))

    return features, labels

In [4]:
# Definition of the lstm model
def lstm_model():
    model = Sequential([
        LSTM(124, input_shape=(14, 360), activation=tf.nn.relu, return_sequences=True),
        Dropout(0.4),
        LSTM(124, activation=tf.nn.relu),
        Dropout(0.3),
        Dense(64, activation=tf.nn.relu),
        Dropout(0.2),
        Dense(2, activation=tf.nn.softmax, name='output_layer')
    ])

    return model

In [5]:
# Function to run the network with training data and testing data
def run_network(model, x_train, y_train, x_test, y_test, iteration, epochs=30):

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=['accuracy'],

    )
    # Train the model
    model.fit(x_train, y_train, epochs=epochs, callbacks=[
        EarlyStopping(patience=8, verbose=1, restore_best_weights=True, monitor='loss')
    ])

    # Get the results
    accuracy, precision, recall, f1, confusion_matrix = utils.get_metrics_keras(model, x_test, y_test,
                                                                                f'{iteration}. LSTM')
    # Return the results as a dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': confusion_matrix
    }

In [6]:
def run_individual(file, particip_num, model, test_size=0.25, epochs=30):
    print('Running ANN for file:', file)
    dataset = np.load(file) # load numpy file containing the preprocessed data for specific participant
    features, labels = dataset['features'], dataset['labels'] # get features and labels from the numpy file

    # transform numpy arrays
    features, labels = transform_dataset(features, labels)
    # split to training and testing data
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed,
                                                        shuffle=True)
    print('X (train) shape:', x_train.shape, 'Y (train) shape:', y_train.shape)
    print('X (test) shape:', x_test.shape, 'Y (test) shape:', y_test.shape)

    return run_network(model, x_train, y_train, x_test, y_test, particip_num, epochs)

In [7]:
i = 1
results = []
for file in files:
    results.append(run_individual(file=file, particip_num=i, model=lstm_model()))
    i += 1
    K.clear_session()

Running ANN for file: dataset_result\P01.npz
X (train) shape: (90, 14, 360) Y (train) shape: (90, 2)
X (test) shape: (30, 14, 360) Y (test) shape: (30, 2)
Epoch 1/30
3/3 [==============================] - 6s 120ms/step - loss: 0.8360 - accuracy: 0.6209
Epoch 2/30
3/3 [==============================] - 0s 112ms/step - loss: 0.9276 - accuracy: 0.5089
Epoch 3/30
3/3 [==============================] - 0s 134ms/step - loss: 0.7684 - accuracy: 0.6197

In [8]:
# Create pandas dataframe with stats from each iterations
df = pd.DataFrame({
    'participant': [x for x in range(1, num_participants + 1)],
    'accuracy': [x['accuracy'] for x in results],
    'precision': [x['precision'] for x in results],
    'recall': [x['recall'] for x in results],
    'f1': [x['f1'] for x in results],
})

df

,participant,accuracy,precision,recall,f1
0,1,0.566667,0.666667,0.555556,0.606061
1,2,0.523810,0.500000,0.600000,0.545455
2,3,0.547619,0.571429,0.380952,0.457143
3,4,0.428571,0.333333,0.200000,0.250000
4,5,0.547619,0.526316,0.500000,0.512821
5,6,0.523810,0.588235,0.434783,0.500000
6,7,0.500000,0.500000,0.476190,0.487805
7,8,0.595238,0.565217,0.650000,0.604651
8,9,0.380952,0.368421,0.333333,0.350000
9,10,0.571429,0.565217,0.619048,0.590909


In [9]:
data_output_folder = 'lstm_individuals'
iteration_data_file_name = 'lstm_individuals_data.xlsx'

os.makedirs(data_output_folder, exist_ok=True)

df.to_excel(os.path.join(data_output_folder, iteration_data_file_name))
'Data from individuals successfully saved.'

'Data from individuals successfully saved.'

In [10]:
# Create a dataframe with statistics
df_stats = pd.DataFrame({
    'average_accuracy': [df['accuracy'].mean()],
    'max_accuracy': [df['accuracy'].max()],
    'accuracy_std': [df['accuracy'].std()],
    'average_precision': [df['precision'].mean()],
    'max_precision': [df['precision'].max()],
    'average_recall': [df['recall'].mean()],
    'max_recall': [df['recall'].max()],
    'average_f1': [df['f1'].mean()],
    'max_f1': [df['f1'].max()],
})

df_stats


,average_accuracy,max_accuracy,accuracy_std,average_precision,max_precision,average_recall,max_recall,average_f1,max_f1
0,0.508995,0.595238,0.063592,0.506379,0.666667,0.485354,0.714286,0.491539,0.638298


In [11]:
# Save the dataframe
stats_file_name = 'lstm_individuals_stats.xlsx'
df_stats.to_excel(os.path.join(data_output_folder, stats_file_name))
'Stats successfully saved.'

'Stats successfully saved.'

In [12]:
utils.print_confusion_matrices(results)

Confusion matrices for the ANN:
[[ 7  5]
 [ 8 10]] 

[[10 12]
 [ 8 12]] 

[[15  6]
 [13  8]] 

[[14  8]
 [16  4]] 

[[13  9]
 [10 10]] 

[[12  7]
 [13 10]] 

[[11 10]
 [11 10]] 

[[12 10]
 [ 7 13]] 

[[ 9 12]
 [14  7]] 

[[11 10]
 [ 8 13]] 

[[11 11]
 [11  9]] 

[[ 9 11]
 [14  8]] 

[[13 10]
 [10  9]] 

[[11  9]
 [11 11]] 

[[10 11]
 [ 6 15]] 

[[ 9 10]
 [13 10]] 

[[ 8 13]
 [10 11]] 

[[13 10]
 [ 9 10]] 

